# Environment

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# !pip install transformers
!pip install datasets
!pip install sentencepiece

In [ ]:
import copy
import io
import json
import math
import os
import time
import re
import random
import numpy as np
import pandas as pd
from datasets import load_dataset, Features, Value, concatenate_datasets, Dataset
from sklearn.model_selection import train_test_split
import torch
from torch.optim import lr_scheduler, AdamW
from torch.utils.data import DataLoader
from tqdm import tqdm, trange
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import get_linear_schedule_with_warmup

# Constant

In [ ]:
"""Basic Setting"""
data_path = "/content/drive/MyDrive/AIcup/data_hpw"
model_path = "/content/drive/MyDrive/AIcup/model_hpw"

"""Data Setting"""
TaskPrefixOriginal = "Private information extraction from patients' records: "
TaskPrefixSliced = "Private information extraction from sliced patients' records: "
TaskPrefixSpliced = "Private information extraction from spliced patients' records: "
PHINull = "PHI:Null"
IgnoredPadIdx = -100
PHINullRatio = 0.3
PhiCategory = ['PATIENT', 'DOCTOR', 'USERNAME', 'PROFESSION',
                'ROOM', 'DEPARTMENT', 'HOSPITAL', 'ORGANIZATION',
                'STREET', 'CITY', 'STATE', 'COUNTRY', 'ZIP', 'LOCATION-OTHER',
                'AGE', 'DATE', 'TIME', 'DURATION', 'SET', 'PHONE', 'FAX', 'EMAIL',
                'URL', 'IPADDR', 'SSN', 'MEDICALRECORD', 'HEALTHPLAN', 'ACCOUNT', 'LICENSE',
                'VEHICLE', 'DEVICE', 'BIOID', 'IDNUM']
"""Model Constant Setting"""
BatchSize = 16
Epochs = int(BatchSize*0.75)
MaxLen = 128
LearningRate = 1e-4
WeightDecay = 0.05
PretrainedModel = "google/flan-t5-large"

# DataReading

In [ ]:
"""Filter PHINull"""
def filter_phi_null_ratio(dataset, ratio):
    # get indices
    phi_null_indices = [i for i, item in enumerate(dataset) if item['label'] == 'PHI:Null']
    other_indices = [i for i, item in enumerate(dataset) if item['label'] != 'PHI:Null']
    print(f"Total {len(dataset)}, PHI null {len(phi_null_indices)}, others {len(other_indices)} ")

    # target null count
    target_phi_null_count = int(min(len(phi_null_indices), len(other_indices) * ratio))
    print("target_phi_null_count", target_phi_null_count)

    if len(phi_null_indices) > target_phi_null_count:
        phi_null_indices = random.sample(phi_null_indices, target_phi_null_count)

    # reunion indices
    final_indices = phi_null_indices + other_indices
    random.shuffle(final_indices)

    final_dataset = dataset.select(final_indices)
    return final_dataset

In [ ]:
"""Add Prefix"""
def add_prefix_to_sample(sample, prefix):
    sample['content'] = prefix + sample['content']
    return sample

def add_prefix_to_dataset(dataset, prefix):
    # 使用 map 函数为每个样本添加前缀
    return dataset.map(lambda x: add_prefix_to_sample(x, prefix))

In [ ]:
def data_reading(data_path1, data_path2, task_prefix):
  # read data
  data1 = load_dataset("csv", data_files=data_path1, delimiter='\t',
                       features = Features({
                          'fid': Value('string'), 'idx': Value('int64'),
                          'content': Value('string'), 'label': Value('string')}),
                       column_names=['fid', 'idx', 'content', 'label'], keep_default_na=False)["train"]
  data2 = load_dataset("csv", data_files=data_path2, delimiter='\t',
                       features = Features({
                          'fid': Value('string'), 'idx': Value('int64'),
                          'content': Value('string'), 'label': Value('string')}),
                       column_names=['fid', 'idx', 'content', 'label'], keep_default_na=False)["train"]
  # add task prefix
  data1 = add_prefix_to_dataset(data1, task_prefix)
  data2 = add_prefix_to_dataset(data2, task_prefix)

  print("pahse1 data:", len(data1))
  print("pahse2 data:", len(data2))
  print("data sample:", data1[200])

  # concatenate data
  data = concatenate_datasets([data1, data2])
  print("data length after cancatenation:", len(data))

  # filter data
  filtered_data = filter_phi_null_ratio(data, PHINullRatio)
  # print filter information
  phi_null_count_after = len([item for item in filtered_data if item['label'] == 'PHI:Null'])
  other_count_after = len(filtered_data) - phi_null_count_after
  phi_null_ratio = phi_null_count_after / other_count_after
  print("After filtering:")
  print("Length after filter:", len(filtered_data))
  print("PHI: NULL count:", phi_null_count_after)
  print("Other labels count:", other_count_after)
  print("PHI: NULL to Other labels ratio:", phi_null_ratio)
  return filtered_data

In [ ]:
# Original data paths
data_org_train_path1 = os.path.join(data_path, "train_phase1_v8_original_train.tsv")
data_org_train_path2 = os.path.join(data_path, "train_phase2_v8_original_train.tsv")
data_org_test_path1 = os.path.join(data_path, "train_phase1_v8_original_test.tsv")
data_org_test_path2 = os.path.join(data_path, "train_phase2_v8_original_test.tsv")

# Sliced data paths
data_slc_train_path1 = os.path.join(data_path, "train_phase1_v8_sliced_train.tsv")
data_slc_train_path2 = os.path.join(data_path, "train_phase2_v8_sliced_train.tsv")
data_slc_test_path1 = os.path.join(data_path, "train_phase1_v8_sliced_test.tsv")
data_slc_test_path2 = os.path.join(data_path, "train_phase2_v8_sliced_test.tsv")

# Spliced data paths
data_spl_train_path1 = os.path.join(data_path, "train_phase1_v8_spliced_train.tsv")
data_spl_train_path2 = os.path.join(data_path, "train_phase2_v8_spliced_train.tsv")
data_spl_test_path1 = os.path.join(data_path, "train_phase1_v8_spliced_test.tsv")
data_spl_test_path2 = os.path.join(data_path, "train_phase2_v8_spliced_test.tsv")

# Read datasets
data_org_train = data_reading(data_org_train_path1, data_org_train_path2, TaskPrefixOriginal)
data_org_test = data_reading(data_org_test_path1, data_org_test_path2, TaskPrefixOriginal)

data_slc_train = data_reading(data_slc_train_path1, data_slc_train_path2, TaskPrefixSliced)
data_slc_test = data_reading(data_slc_test_path1, data_slc_test_path2, TaskPrefixSliced)

data_spl_train = data_reading(data_spl_train_path1, data_spl_train_path2, TaskPrefixSpliced)
data_spl_test = data_reading(data_spl_test_path1, data_spl_test_path2, TaskPrefixSpliced)

In [ ]:
train_data = concatenate_datasets([data_org_train, data_slc_train, data_spl_train])
print("total train data:", len(train_data))
train_data = train_data.shuffle(seed=42)
# train_data = train_data.select(range(30000))
# print("sampled train data:", len(train_data))
for i in range(100, 103):
  print("train data samples:", train_data[i])

test_data = concatenate_datasets([data_org_test, data_slc_test, data_spl_test])
test_data = test_data.shuffle(seed=42)
print("total test data:", len(test_data))
# test_data = test_data.select(range(3000))
# print("sampled test data:", len(test_data))
for i in range(100, 103):
  print("test data samples:", test_data[i])

# ModelConfig

In [ ]:
"""Model Config"""
tokenizer = T5Tokenizer.from_pretrained(PretrainedModel)

model = T5ForConditionalGeneration.from_pretrained(PretrainedModel)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# DataloaderTrain

In [ ]:
"""Tokenizer Template"""
def collate_batch_with_prompt_template(batch, tokenizer, IGNORED_PAD_IDX=IgnoredPadIdx):
    texts = [data['content'] for data in batch]
    encoded_seq = tokenizer(texts, padding="max_length", truncation=True, max_length=MaxLen, return_tensors="pt")

    labels = [data['label'] for data in batch]
    encoded_label = tokenizer(labels, padding="max_length", truncation=True, max_length=MaxLen, return_tensors="pt")['input_ids']
    encoded_label[encoded_label == tokenizer.pad_token_id] = IGNORED_PAD_IDX

    return encoded_seq['input_ids'], encoded_seq['attention_mask'], encoded_label

In [ ]:
"""Dataloader"""
# train dataloader
train_dataloader = DataLoader(train_data,
                              batch_size=BatchSize,
                              collate_fn=lambda batch: collate_batch_with_prompt_template(batch, tokenizer),
                              shuffle=True,
                              drop_last=True)
# dev dataloader
test_dataloader = DataLoader(test_data,
                             batch_size=1,
                             collate_fn=lambda batch: collate_batch_with_prompt_template(batch, tokenizer),
                             shuffle=False,
                             drop_last=True)

dataloaders = {"train": train_dataloader, "test": test_dataloader}
print(len(train_dataloader))
print(len(test_dataloader))

In [ ]:
"""Test Train_dataloader"""
titer = iter(train_dataloader)
tks, masks, labels = next(titer)
print(tks.shape)
print(tks[0])
print()
print(masks.shape)
print(masks[0])
print()
print(labels.shape)
print(labels[0])
print()

# Optimizer

In [ ]:
"""optimizer config"""
optimizer = AdamW(model.parameters(), lr=LearningRate, weight_decay=WeightDecay)

# steps calculation
num_training_steps = len(dataloaders["train"])*Epochs
CountSteps = int(num_training_steps*0.1/Epochs) // 10 *10*(16//BatchSize)
WarmUpSteps = int(num_training_steps*0.01)
print(f"num_training_steps {num_training_steps}, warn_up_steps {WarmUpSteps}, count each {CountSteps} steps")

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=WarmUpSteps,
    num_training_steps=num_training_steps
)

# Training

In [ ]:
def write_losses(loss_path, train_losses, dev_losses):
  losses={}
  losses["train"] = train_losses
  losses["test"] = dev_losses
  with open(loss_path, "w") as out_config:
    json.dump(losses, out_config, indent=4)

In [ ]:
"""Train Model"""
# model path
name = str(int(time.time()))
save_path = os.path.join(model_path, name)
model_name = save_path + f"/best_{name}.pt"

loss_path = os.path.join(save_path, "loss.csv")

if not os.path.isdir(save_path):
    os.mkdir(save_path)

best_loss = float('inf')
train_losses = []
test_losses = []

for epoch in range(Epochs):
    print("[Training] Epoch {}/{}".format(epoch, Epochs - 1))
    print("-" * 10)

    running_loss_train = 0.0
    running_loss_test = 0.0
    intermediate_loss = 0.0

    # Training phase
    model.train()
    for count, (seqs, masks, labels) in enumerate(tqdm(dataloaders["train"])):
        seqs, labels, masks = seqs.to(device), labels.to(device), masks.to(device)

        optimizer.zero_grad()

        outputs = model(input_ids=seqs, labels=labels, attention_mask=masks)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        scheduler.step()

        running_loss_train += loss.item()
        intermediate_loss += loss.item()
        # Print training losses
        if count % CountSteps == 0 and count != 0:
            average_loss = intermediate_loss / CountSteps
            print(f"Loss in epoch{epoch}-step{count}: {average_loss:.4f}")
            train_losses.append(average_loss)
            intermediate_loss = 0

    # Print epoch train losses
    epoch_train_loss = running_loss_train / len(dataloaders["train"])
    print(f"[Training] Train Loss: {epoch_train_loss:.4f}")

    # Testing phase
    model.eval()
    with torch.no_grad():
        for seqs_test, masks_test, labels_test in tqdm(dataloaders["test"]):
            seqs_test, labels_test, masks_test = seqs_test.to(device), labels_test.to(device), masks_test.to(device)
            outputs = model(input_ids=seqs_test, labels=labels_test, attention_mask=masks_test)
            # loss = outputs.loss.mean()
            loss = outputs.loss
            running_loss_test += loss.item()

    # print epoch test loss
    epoch_test_loss = running_loss_test / len(dataloaders["test"])
    print(f"[Training] Test Loss: {epoch_test_loss:.4f}")

    # save the best model
    test_losses.append(epoch_test_loss)

    if epoch_test_loss < best_loss:
        # write losses in each epoch
        write_losses(loss_path, train_losses, test_losses)
        best_loss = epoch_test_loss
        best_model_wts = copy.deepcopy(model.state_dict())
        torch.save(model.state_dict(), model_name)
        print(f"[INFO] Updated best model on dev checkpoint: {model_name}")

# DataloaderVal

In [ ]:
"""Add Prefix"""
def add_prefix_to_sample(sample, prefix):
    sample['content'] = prefix + sample['content']
    return sample

def add_prefix_to_dataset(dataset, prefix):
    return dataset.map(lambda x: add_prefix_to_sample(x, prefix))

In [ ]:
def reading_validation_data(val_path, task_prefix):
  # read data
  val_data = load_dataset("csv", data_files=val_path, delimiter='\t',
                        features = Features({
                            'fid': Value('string'), 'idx': Value('int64'),
                            'content': Value('string'), 'label': Value('string')}),
                        column_names=['fid', 'idx', 'content', 'label'])["train"]

  # add prefix to each dataset
  val_data = add_prefix_to_dataset(val_data, task_prefix)
  print(len(val_data))
  print("data sample:", val_data[20])
  return val_data

In [ ]:
"""Tokenizer Template"""
def collate_batch_with_prompt_template_val(batch, tokenizer, ):
    texts = [data['content'] for data in batch]
    fids = [data['fid'] for data in batch]
    pos = [data['idx'] for data in batch]
    contents = [data['content'] for data in batch]
    encoded_seq = tokenizer(texts, padding="max_length", truncation=True, max_length=MaxLen, return_tensors="pt")
    return encoded_seq['input_ids'], encoded_seq['attention_mask'], fids, pos, contents

In [ ]:
"""Get validation datasets"""
val_path_org = os.path.join(data_path, "valid_phase1_v8_original.tsv")
val_path_slc = os.path.join(data_path, "valid_phase1_v8_sliced.tsv")
val_path_spl = os.path.join(data_path, "valid_phase1_v8_spliced.tsv")

val_data_org = reading_validation_data(val_path_org, TaskPrefixOriginal)
val_data_slc = reading_validation_data(val_path_slc, TaskPrefixSliced)
val_data_spl = reading_validation_data(val_path_spl, TaskPrefixSpliced)

# Concatenate datasets
val_data = concatenate_datasets([val_data_org, val_data_slc, val_data_spl])
val_data = list(val_data)
# val_data = list(val_data)[:100]
print("validation length:", len(val_data))
for i in range(100, 103):
  print(val_data[i])

In [ ]:
ValBatchSize = 128
val_dataloader = DataLoader(val_data,
                            batch_size=ValBatchSize,
                            collate_fn=lambda batch: collate_batch_with_prompt_template_val(batch, tokenizer),
                            shuffle=False)

# Generation


In [ ]:
"""model reload"""
name = "1701317300_flant5_v1_large_task1"
time = name[:10]
model_name = f"drive/MyDrive/AIcup/model_hpw/{name}/best_{time}.pt"
answer_path = f"drive/MyDrive/AIcup/model_hpw/{name}/answer_{time}.txt"
prediction_path = f"drive/MyDrive/AIcup/model_hpw/{name}/prediction_{time}.txt"
model.load_state_dict(torch.load(model_name))
model = model.to(device)

In [ ]:
def remove_prefixes(text):
    if TaskPrefixOriginal in text:
        text = text.replace(TaskPrefixOriginal, "")
    if TaskPrefixSliced in text:
        text = text.replace(TaskPrefixSliced, "")
    if TaskPrefixSpliced in text:
        text = text.replace(TaskPrefixSpliced, "")
    return text

In [ ]:
with open(os.path.join(prediction_path), 'w', encoding='utf8') as f_predictions:
  for seqs, masks, fids, pos, contents in tqdm(val_dataloader):
    contents = list(map(remove_prefixes, contents))
    with torch.no_grad():
      seqs, masks = seqs.to(device), masks.to(device)
      predicted_tokens = model.generate(input_ids=seqs, attention_mask=masks)
      predicted_tokens = torch.squeeze(predicted_tokens)
      predicted_strings = tokenizer.batch_decode(predicted_tokens, skip_special_tokens=True)
      for idx, pred in enumerate(predicted_strings):
        if PHINull in pred.strip():
          continue
        # print(f'{fids[idx]}\t{pos[idx]}\t{contents[idx]}\t{pred}')
        f_predictions.write(f'{fids[idx]}\t{pos[idx]}\t{contents[idx]}\t{pred}\n')